In [1]:
import warnings
import time
import itertools
from typing import Dict, List

import requests
from fake_headers import Headers
from bs4 import BeautifulSoup
from returns.io import IO
from lxml import html

'''
Общий алгоритм работы
---------------------
-> Определь неизменяемые входные данные ({url, domain, headers{browser,os,headers}}).
-> Получить общее количество страниц пагинации с данными (товарами)
-> Получить url всех страниц пагинации (в случае e-katalog это порядковый номер страницы)
-> Получить список списков со всеми url конкретного товарова
-> Распаковать все списки в один список.
-> Добавить ко всем url домен, как итог список окончательных ссылок к товару.
_____________________

In [2]:
warnings.filterwarnings('ignore')

In [3]:
class imdict(dict):
    def __hash__(self):
        return id(self)

    def _immutable(self, *args, **kws):
        raise TypeError('object is immutable')

    __setitem__ = _immutable
    __delitem__ = _immutable
    clear       = _immutable
    update      = _immutable
    setdefault  = _immutable
    pop         = _immutable
    popitem     = _immutable

In [4]:
data = imdict({'URL':'https://www.e-katalog.ru/list/190/',
               'DOMAIN': 'https://www.e-katalog.ru',
               'headers': {'browser':"chrome", 
                          'os':"win",
                          'headers': True}})

In [5]:
def generate_headers(headers_setting: Dict) -> IO[Dict]:
    '''
    Генерирует html заголовки запросов. В данном случае фейковые.
    '''
    #TODO Расширить количество аргументов.

    return Headers(
                   browser = headers_setting['browser'],
                   os = headers_setting['os'],
                   headers = headers_setting['headers']
                   ).generate()

In [6]:
def get_page_html(url: str, verify: bool = False):
    '''На основе url получает всю html страницы, без потверждения ssl(опционально)'''

    return BeautifulSoup(requests.get(url, verify=verify).content, 'html.parser')

In [7]:
def get_product_amount_page(url: str, headers: Dict = generate_headers(data['headers'])) -> IO[int]:
    '''
    Возвращает общее количество страниц для конкретного типа продукта
    | ---
    | url = базовый url страницы конкретного продукта (data['url'])
    | headers = Шапка html запроса в формате словаря.
    | ---
    '''

    def page_count(tree):
        return tree.xpath('//div[@class="ib page-num"]//a[last()]/text()')   

    return int(page_count(html.fromstring(requests.get(url, headers = headers).content))[0])

In [8]:
def collect_product_pages(url: str, pages_count: int) -> List[str]:
    '''
    *Получает список url страниц с товарами.*
    | ---
    | url = базовый url страницы конкретного продукта (data['url'])
    | pages_count = Общее количество страниц для продукта.
    | ---
    '''
    return list(map(lambda x: url + f"{x}/", range(1, pages_count)))

In [9]:
def collect_url_from_page(page_url: str, headers = generate_headers(data['headers']), sleep: int = 1) -> List[str]:
    '''
    *Получает все url продуктов со страницы(получает список страницы), возвращает список url*
    | ---
    | page_url = url страницы со товармаи
    | headers = Шапка html запроса в формате словаря
    | sleep = Время задержки после запроса
    | ---
    '''

    def get_product_url(tree):
        return tree.xpath("//a[@class='model-short-title no-u no-u']/@href") 
    
    time.sleep(sleep)
    return get_product_url(html.fromstring(requests.get(page_url, headers=headers).content))
    

In [11]:
pages_urls = collect_product_pages(data['URL'], get_product_amount_page(
                                                          data['URL'],
                                                          generate_headers(data['headers'])))

In [12]:
all_url_product = list(itertools.chain(*list(map(collect_url_from_page,urls)))) # Распоковка всех списков

In [14]:
full_url_path_to_product = list((map(lambda x: data['DOMAIN'] + x, all_urls)))

['https://www.e-katalog.ru/WD-WD141KRYZ.htm',
 'https://www.e-katalog.ru/WD-WUH721414ALE6L4.htm',
 'https://www.e-katalog.ru/TOSHIBA-HDTP220EK3CA.htm',
 'https://www.e-katalog.ru/TOSHIBA-HDWT380UZSVA.htm',
 'https://www.e-katalog.ru/SEAGATE-ST8000NM0055.htm',
 'https://www.e-katalog.ru/WD-WD20EFZX.htm',
 'https://www.e-katalog.ru/TRANSCEND-TS4TSJ25H3P.htm',
 'https://www.e-katalog.ru/SEAGATE-ST18000NM000J.htm',
 'https://www.e-katalog.ru/TOSHIBA-MG08ACA16TE.htm',
 'https://www.e-katalog.ru/SEAGATE-ST10000VE0008.htm',
 'https://www.e-katalog.ru/SEAGATE-ST1000LM048.htm',
 'https://www.e-katalog.ru/TOSHIBA-HDWL110EZSTA.htm',
 'https://www.e-katalog.ru/TRANSCEND-TS4TSJ25M3S.htm',
 'https://www.e-katalog.ru/WD-WD121KFBX.htm',
 'https://www.e-katalog.ru/WD-WD30EZRX.htm',
 'https://www.e-katalog.ru/TOSHIBA-HDTP340EK3CA.htm',
 'https://www.e-katalog.ru/TOSHIBA-MG07SCA14TE.htm',
 'https://www.e-katalog.ru/TOSHIBA-HDWN180EZSTA.htm',
 'https://www.e-katalog.ru/WD-HUS726T4TALE6L4.htm',
 'https://w